# Export Results

In [1]:
%load_ext autoreload
%autoreload 2
%pdb on

Automatic pdb calling has been turned ON


In [2]:
import os, sys
from pathlib import Path
from random import *
from datetime import *
from IPython.core.debugger import set_trace
from tqdm import tqdm
import warnings
from omegaconf import OmegaConf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from shapely.geometry import *
from rasterio.errors import NotGeoreferencedWarning

warnings.filterwarnings("ignore", category=NotGeoreferencedWarning)
import rasterio

from einops import rearrange
import kornia.augmentation as K

import torch
from torchmetrics.classification import MulticlassJaccardIndex
from torchgeo.trainers import SemanticSegmentationTask

/home/akramzaytar/ssdshared/akramz_datasets/config/miniforge3/envs/wtlv2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dataloaders.dfc2022_datamodule import DFC2022DataModule
from dataloaders.isprs_datamodule import ISPRSDataModule

In [4]:
dataset = "potsdam"
method_fn = "diversityvit"

In [5]:
method_name = f"{dataset} {method_fn}"
config_file = f"jobs/{dataset}.yaml"
gpu = 1
device = f"cuda:{gpu}"

# Get the model directories
model_dirs = list(
    Path("./internal_checkpoints").glob(method_name.lower().replace(" ", "_") + "_*")
)
percent2models = {}
for model_dir in model_dirs:
    percent = int(model_dir.name.split("_")[-2])
    if percent not in percent2models:
        percent2models[percent] = list()
    ckpt_files = list(model_dir.glob("*.ckpt"))
    ckpt_files = sorted(ckpt_files, key=lambda x: x.stat().st_mtime, reverse=True)
    if ckpt_files:
        percent2models[percent].append(ckpt_files[0])

# Remove empty percent2models
percent2models = {k: v for k, v in percent2models.items() if v}

In [6]:
for percentage in [1, 5, 10, 25, 50]:

    if not percentage in percent2models:
        continue

    print(f"{method_name} ({percentage}%) ---->")
    models = percent2models[percentage]

    for ckpt in models:

        # pretend that we are working with a single file
        config = OmegaConf.load(config_file)
        config.evaluation.method_name = method_name
        config.datamodule.train_coordinate_file_path = f"/home/akramzaytar/ssdshared/akramz_datasets/wtlv2/submissions/{dataset}/{method_fn}.txt"
        config.trainer.devices = [gpu]
        config.datamodule.root_dir = str(Path.home() / config.datamodule.root_dir)

        # Load model
        task = SemanticSegmentationTask.load_from_checkpoint(
            ckpt, weights=True
        )  # A hack
        task = task.to(device).half()
        task.eval()

        # Load datamodule and dataloader
        if config.datamodule.dataset == "DFC2022":
            datamodule = DFC2022DataModule(**config.datamodule)
            datamodule.setup()
            dataloader = datamodule.test_dataloader()  # batch size is 1
            pad = K.PadTo(size=(2048, 2048), pad_mode="constant", pad_value=0.0)
            indices = (
                torch.Tensor([1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14]).int().to(device)
            )  # 12 classes

        else:  # vaihingen and potsdam
            datamodule = ISPRSDataModule(**config.datamodule)
            datamodule.setup()
            dataloader = datamodule.test_dataloader()  # batch size is 1
            pad = K.PadTo(size=(3840, 3840), pad_mode="constant", pad_value=0.0)
            p_pad = K.PadTo(size=(6016, 6016), pad_mode="constant", pad_value=0.0)
            indices = torch.Tensor([0, 1, 2, 3, 4, 5]).int().to(device)

        jaccard = torch.zeros(6).to(device)
        jaccard_metric = MulticlassJaccardIndex(
            num_classes=6,
            ignore_index=-1,
            average="none",
        ).to(device)
        torch.cuda.empty_cache()

        for i, batch in enumerate(dataloader):
            x = batch["image"].to(device).half()
            height, width = x.shape[-2:]
            if (
                config.datamodule.dataset == "DFC2022"
                or config.datamodule.dataset == "vaihingen"
            ):
                x = pad(x)
            else:
                x = p_pad(x)
            preds = task(x)
            tmp_preds = preds
            preds = preds[0, :, :height, :width]
            preds = rearrange(preds, "c h w -> (h w) c")
            target = batch["mask"].to(device).flatten()
            jaccard += jaccard_metric(preds, target)
            del [x, preds, target, tmp_preds]
            torch.cuda.empty_cache()

        ave_jac = jaccard / len(dataloader)
        ave_jac_specific_classes = torch.index_select(ave_jac, 0, indices)
        scores = [float(torch.mean(ave_jac_specific_classes).cpu().numpy())] + [
            round(e, 5) for e in ave_jac_specific_classes.cpu().numpy().tolist()
        ]
        print(" ".join(map(str, scores)))

potsdam diversityvit (1%) ---->


/home/akramzaytar/ssdshared/akramz_datasets/config/miniforge3/envs/wtlv2/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


0.524730384349823 0.62951 0.54732 0.59305 0.58652 0.61751 0.17447
0.6081923842430115 0.73194 0.76181 0.62475 0.63225 0.71216 0.18624
potsdam diversityvit (5%) ---->
0.6053739786148071 0.74946 0.78055 0.62973 0.6291 0.70736 0.13604
0.6651648283004761 0.79051 0.85004 0.65716 0.70154 0.75366 0.23807
potsdam diversityvit (10%) ---->
0.6291732788085938 0.75864 0.82048 0.64211 0.67823 0.74181 0.13377
0.5777608156204224 0.75275 0.74386 0.60308 0.542 0.69007 0.1348
potsdam diversityvit (25%) ---->
0.5769183039665222 0.72538 0.7559 0.63139 0.66204 0.58525 0.10155
0.5991523265838623 0.72396 0.76751 0.63969 0.65981 0.65751 0.14643
potsdam diversityvit (50%) ---->
0.6650083065032959 0.77227 0.81546 0.67007 0.69159 0.75852 0.28215
0.5896179676055908 0.70507 0.77434 0.63862 0.60585 0.71395 0.09988


---